### Testing Playground for P0 Programs

In [ ]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString
from ST import printSymTab

def runwasm(wasmfile):
    from IPython.core.display import display, Javascript
    display(Javascript("""
    const params = { 
        P0lib: { 
            write: i => this.append_stream({text: '' + i, name: 'stdout'}),
            writeln: () => this.append_stream({text: '\\n', name: 'stdout'}),
            read: () => window.prompt()
        }
    }

    fetch('""" + wasmfile + """') // asynchronously fetch file, return Response object
      .then(response => response.arrayBuffer()) // read the response to completion and stores it in an ArrayBuffer
      .then(code => WebAssembly.compile(code)) // compile (sharable) code.wasm
      .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
    // .then(instance => instance.exports.program()); // run the main program; not needed if start function specified
    """))

Write, compile and execute P0 programs for testing using the following steps:
1. Define a string constant containing a P0 program to be validated in the cell below.
2. Update the `compileString` parameter to accept the string constant in the cell below.
3. Select `Kernel > Restart & Run All` to view and execute the target WebAssembly program.

In [ ]:
# Simple array literal assignment
test1 = """
program p
    var a: [1..3] → integer
    a := [3, 7, 11]
    write(a[1])
    write(a[2])
    write(a[3])
"""

# Simple array interval assignment
test2 = """
program p
    var a: [1..3] → integer
    a := [6..8]
    write(a[1])
    write(a[2])
    write(a[3])
"""

# Multiple assignment with array literal at start
test3 = """
program p
    var a: [1..3] → integer
    var b: integer
    var c: boolean
    a, b, c := [3, 7, 11], 42, true
    write(a[1])
    write(a[2])
    write(a[3])
    write(b)
    if c then write(100)
"""

# Multiple assignment with array literal in middle
test4 = """
program p
    var a: [1..3] → integer
    var b: integer
    var c: boolean
    b, a, c := 42, [3, 7, 11], true
    write(a[1])
    write(a[2])
    write(a[3])
    write(b)
    if c then write(100)
"""

# Array literal with mixed types
test5 = """
program p
    var a: [1..3] → integer
    a := [3, true, 11]
"""

# Array literal with wrong type
test6 = """
program p
    var a: [1..3] → integer
    a := [false, true, false]
"""

# Array with non-integer range
test7 = """
program p
    var a: [1..3] → integer
    a := [true..8]
"""

# Array with non-increasing range
test8 = """
program p
    var a: [1..3] → integer
    a := [6..5]
"""

# Array literal of boolean values
test9 = """
program p
    var a: [1..3] → boolean
    a := [false, true, false]
    if a[1] = true then write(100)
    if a[2] = true then write(200)
    if a[3] = true then write(300)
"""

# Single array index assignment (regression test)
test10 = """
program p
    var a: [1..3] → integer
    a[2] := 42
    write(a[2])
"""

# Array direct comparisons
test11 = """
program p
    var a: [1..3] → integer
    var b: [1..3] → integer
    a := [3, 7, 11]
    b := [3, 8, 11]
    if a = [3, 7, 11] then write(100)
    if a = [3, 8, 11] then write(200)
    if [3, 7, 11] = [3, 7, 11] then write(300)
    if [3, 7, 11] = [3, 8, 11] then write(400)
    if a = a then write(500)
    if a = b then write(600)
    if [1, 2, 3] = [1, 2, 3, 4] then write(700)
    if [1, 2, 3, 4] = [1, 2, 3] then write(800)
"""

# Indexing array literals
test12 = """
program p
    write([2,3,4][0])
    write([2,3,4][1])
    write([2,3,4][2])
"""

# Assigning subarrays
test13 = """
program p
    var a: [1..5] → integer
    var b: [1..3] → integer
    a := [103..107]
    b := a[1:4]
    write(b[1])
    write(b[2])
    write(b[3])
"""

# Comparing subarrays
test14 = """
program p
    var a: [1..5] → integer
    var b: [1..5] → integer
    a := [1,2,3,4,5]
    b := [1,2,3,4,5]
    if a[1:4] = b[1:4] then write(100)
    if a[1:4] = b[2:5] then write(200)
    if a[1:4][0:2] = b[1:4][0:2] then write(300)
    if a[1:4][0:2] = b[2:5][0:2] then write(400)
"""

# Array variable index below lower bound
test15 = """
program p
    var a: [1..3] → integer
    a := [6..8]
    write(a[0])
"""

# Array variable index at lower bound
test16 = """
program p
    var a: [1..3] → integer
    a := [6..8]
    write(a[1])
"""

# Array variable index at upper bound
test17 = """
program p
    var a: [1..3] → integer
    a := [6..8]
    write(a[3])
"""

# Array variable index above upper bound
test18 = """
program p
    var a: [1..3] → integer
    a := [6..8]
    write(a[4])
"""

# Subarray index below lower bound
test19 = """
program p
    var a: [1..5] → integer
    a := [6..10]
    write(a[1:4][-1])
"""

# Subarray index at lower bound
test20 = """
program p
    var a: [1..5] → integer
    a := [6..10]
    write(a[1:4][0])
"""

# Subarray index at upper bound
test21 = """
program p
    var a: [1..5] → integer
    a := [6..10]
    write(a[1:4][2])
"""

# Subarray index above upper bound
test22 = """
program p
    var a: [1..5] → integer
    a := [6..10]
    write(a[1:4][3])
"""

# Array literal index below lower bound
test23 = """
program p
    var a: [1..5] → integer
    a := [6..10]
    write([6..8][-1])
"""

# Array literal index at lower bound
test24 = """
program p
    var a: [1..5] → integer
    a := [6..10]
    write([6..8][0])
"""

# Array literal index at upper bound
test25 = """
program p
    var a: [1..5] → integer
    a := [6..10]
    write([6..8][2])
"""

# Array literal index above upper bound
test26 = """
program p
    var a: [1..5] → integer
    a := [6..10]
    write([6..8][3])
"""

# Array variable subarray covering full range
test27 = """
program p
    var a: [1..3] → integer
    var b: [1..3] → integer
    a := [6..8]
    b := a[1:4]
    write(b[1])
    write(b[2])
    write(b[3])
"""

# Array variable subarray below lower bound
test28 = """
program p
    var a: [1..3] → integer
    var b: [1..3] → integer
    a := [6..8]
    b := a[0:4]
"""

# Array variable subarray above upper bound
test29 = """
program p
    var a: [1..3] → integer
    var b: [1..3] → integer
    a := [6..8]
    b := a[1:5]
"""

# Subarray of subarray covering full range
test30 = """
program p
    var a: [1..5] → integer
    var b: [1..3] → integer
    a := [1,2,3,4,5]
    b := a[1:5][0:3]
    write(b[1])
    write(b[2])
    write(b[3])
"""

# Subarray of subarray below lower bound
test31 = """
program p
    var a: [1..5] → integer
    var b: [1..5] → integer
    a := [1,2,3,4,5]
    b := a[1:5][-1:3]
"""

# Subarray of subarray above upper bound
test32 = """
program p
    var a: [1..5] → integer
    var b: [1..5] → integer
    a := [1,2,3,4,5]
    b := a[1:5][0:4]
"""

# Array literal subarray covering full range
test33 = """
program p
    var a: [1..3] → integer
    a := [6..8][0:3][0:3]
    write(a[1])
    write(a[2])
    write(a[3])
"""

# Array literal subarray below lower bound
test34 = """
program p
    var a: [1..3] → integer
    a := [6..8][0:3][-1:3]
"""

# Array literal subarray above upper bound
test35 = """
program p
    var a: [1..3] → integer
    a := [6..8][0:3][0:4]
"""

# Update this line to run a specific testcase
compileString(test1, "test.wat")

In [ ]:
!cat -n "test.wat"

In [ ]:
!wat2wasm --enable-bulk-memory test.wat || rm test.wasm

In [ ]:
runwasm("test.wasm")